In [1]:
library ('tidyverse')
library ('irr')
#Lin's concordance correlation coefficient (CCC) is the concordance
library('epiR')
library('DescTools')

-- Attaching packages --------------------------------------- tidyverse 1.3.0 --

v ggplot2 3.3.2     v purrr   0.3.4
v tibble  3.0.3     v dplyr   1.0.1
v tidyr   1.1.1     v stringr 1.4.0
v readr   1.3.1     v forcats 0.5.0

-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()

Loading required package: lpSolve

Warning message:
"package 'epiR' was built under R version 4.0.3"
Loading required package: survival

Package epiR 1.0-15 is loaded

Type help(epi.about) for summary information

Type browseVignettes(package = 'epiR') to learn how to use epiR for applied epidemiological analyses




Warning message:
"package 'DescTools' was built under R version 4.0.3"


## Download data

In [2]:
path_analysis = "D:\\vm_exchange\\PhD_Data\\HealthAssessment"
#df = read_delim(file.path(path_analysis,"HA_all.csv"), ";", escape_double = FALSE, trim_ws = TRUE)
df = read_delim(file.path(path_analysis,"HA_all_20-10-2020V3.csv"), ";", escape_double = FALSE, trim_ws = TRUE)
#convert into to correct format
c_fac = c('PassageID', 'BirdID', 'Diarrhea', 'MissingToes','RFBumblefoot')
df[c_fac] = lapply(df[c_fac], factor)
c_cont = c('CombPeck','CombColor','FeatherNeck','FeatherWing','FeatherTail','FeatherCloaca','CloacaWounds',
            'RFPododermatitis','RFinjuries')
#for (v in c_cont){
#    df[[v]] = as.double(gsub(",", ".", df[[v]]))}
#order with respect to the birdID to ensure comparing the correct birds together
df %>% arrange(desc(Person))
#ATTENTION: verify you have both the same number fo birds manually in the csv file (cant be bother doing this in R)
str(df) #??

Parsed with column specification:
cols(
  Person = col_character(),
  PassageID = col_double(),
  BirdID = col_character(),
  CombPeck = col_double(),
  CombColor = col_double(),
  FeatherNeck = col_double(),
  FeatherWing = col_double(),
  FeatherTail = col_double(),
  FeatherCloaca = col_double(),
  CloacaWounds = col_double(),
  Diarrhea = col_double(),
  MissingToes = col_double(),
  RFPododermatitis = col_double(),
  RFBumblefoot = col_double(),
  RFinjuries = col_double(),
  LFPododermatitis = col_double(),
  LFBumblefoot = col_double(),
  LFinjuries = col_double()
)



Person,PassageID,BirdID,CombPeck,CombColor,FeatherNeck,FeatherWing,FeatherTail,FeatherCloaca,CloacaWounds,Diarrhea,MissingToes,RFPododermatitis,RFBumblefoot,RFinjuries,LFPododermatitis,LFBumblefoot,LFinjuries
<chr>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>
Mark,1,16 gris,200,0,125,0,25,0,0,0,0,50,0,0,25,0,0
Mark,2,16 gris,200,0,25,0,0,0,0,0,0,50,0,0,50,0,0
Mark,1,22 vert,200,0,50,0,0,0,0,0,0,75,0,25,50,0,25
Mark,2,22 vert,200,0,25,0,25,0,0,0,0,50,0,25,50,0,25
Mark,1,26 vert,175,0,25,0,0,0,0,0,0,0,0,25,25,0,0
Mark,2,26 vert,125,0,0,0,0,0,0,0,0,0,0,25,25,0,0
Mark,1,32 noir,200,0,0,0,0,0,0,0,0,50,0,0,50,0,0
Mark,2,32 noir,200,0,0,0,0,0,0,0,0,50,0,0,75,0,0
Mark,1,39 vert,200,0,115,0,0,0,0,0,0,100,0,0,100,0,0


tibble [72 x 18] (S3: spec_tbl_df/tbl_df/tbl/data.frame)
 $ Person          : chr [1:72] "Mark" "Camille" "Mark" "Camille" ...
 $ PassageID       : Factor w/ 2 levels "1","2": 1 1 2 2 1 1 2 2 1 1 ...
 $ BirdID          : Factor w/ 18 levels "16 gris","22 vert",..: 1 1 1 1 2 2 2 2 3 3 ...
 $ CombPeck        : num [1:72] 200 200 200 200 200 200 200 200 175 100 ...
 $ CombColor       : num [1:72] 0 0 0 0 0 0 0 0 0 0 ...
 $ FeatherNeck     : num [1:72] 125 75 25 100 50 50 25 0 25 0 ...
 $ FeatherWing     : num [1:72] 0 0 0 0 0 0 0 0 0 0 ...
 $ FeatherTail     : num [1:72] 25 0 0 0 0 15 25 0 0 0 ...
 $ FeatherCloaca   : num [1:72] 0 0 0 0 0 15 0 15 0 0 ...
 $ CloacaWounds    : num [1:72] 0 0 0 0 0 0 0 0 0 0 ...
 $ Diarrhea        : Factor w/ 1 level "0": 1 1 1 1 1 1 1 1 1 1 ...
 $ MissingToes     : Factor w/ 1 level "0": 1 1 1 1 1 1 1 1 1 1 ...
 $ RFPododermatitis: num [1:72] 50 0 50 75 75 100 50 100 0 25 ...
 $ RFBumblefoot    : Factor w/ 2 levels "0","50": 1 1 1 1 1 1 1 1 1 1 ...
 $ RFinj

In [3]:
df %>% arrange(desc(BirdID))
df

Person,PassageID,BirdID,CombPeck,CombColor,FeatherNeck,FeatherWing,FeatherTail,FeatherCloaca,CloacaWounds,Diarrhea,MissingToes,RFPododermatitis,RFBumblefoot,RFinjuries,LFPododermatitis,LFBumblefoot,LFinjuries
<chr>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>
Mark,1,74 noir,200,0,0,0,0,0,50,0,0,100,0,0,25,0,0
Camille,1,74 noir,200,0,0,0,0,0,0,0,0,100,0,0,0,0,0
Camille,2,74 noir,200,0,0,0,0,0,0,0,0,100,0,0,50,0,0
Mark,2,74 noir,200,0,0,0,25,0,0,0,0,100,0,0,25,0,0
Mark,1,67 gris,175,0,0,0,0,0,0,0,0,75,0,0,75,0,0
Camille,1,67 gris,175,0,0,0,25,0,0,0,0,100,0,0,100,0,0
Mark,2,67 gris,175,0,0,0,25,0,0,0,0,75,0,0,50,0,0
Camille,2,67 gris,150,0,0,0,15,0,0,0,0,100,0,0,100,0,0
Mark,1,63 blanc,200,0,25,0,25,0,0,0,0,50,0,0,100,0,0


Person,PassageID,BirdID,CombPeck,CombColor,FeatherNeck,FeatherWing,FeatherTail,FeatherCloaca,CloacaWounds,Diarrhea,MissingToes,RFPododermatitis,RFBumblefoot,RFinjuries,LFPododermatitis,LFBumblefoot,LFinjuries
<chr>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>
Mark,1,16 gris,200,0,125,0,25,0,0,0,0,50,0,0,25,0,0
Camille,1,16 gris,200,0,75,0,0,0,0,0,0,0,0,0,0,0,0
Mark,2,16 gris,200,0,25,0,0,0,0,0,0,50,0,0,50,0,0
Camille,2,16 gris,200,0,100,0,0,0,0,0,0,75,0,0,75,0,0
Mark,1,22 vert,200,0,50,0,0,0,0,0,0,75,0,25,50,0,25
Camille,1,22 vert,200,0,50,0,15,15,0,0,0,100,0,0,75,0,0
Mark,2,22 vert,200,0,25,0,25,0,0,0,0,50,0,25,50,0,25
Camille,2,22 vert,200,0,0,0,0,15,0,0,0,100,0,15,50,0,15
Mark,1,26 vert,175,0,25,0,0,0,0,0,0,0,0,25,25,0,0


# Intra observer reliability

In [4]:
#create a file to register data
sink(file.path(path_analysis,'RealiabilityIntra.txt'))

In [5]:
#continuous data with ICC & CCC
for (person in unique(df[['Person']])){
    if (is.na(person)==FALSE){
        cat(paste0('--------------------------------------------------\n', person),
            file=file.path(path_analysis,'RealiabilityIntra.txt'), append=TRUE)
        for (var in c_cont){
            cat(paste0('-----------------\n',var), file=file.path(path_analysis,'RealiabilityIntra.txt'), append=TRUE)
            x=subset(df, (df$PassageID =="1") & (df$Person==person))[[var]]
            y=subset(df, (df$PassageID =="2") & (df$Person==person))[[var]]
            cat(paste0('------------ ICC'), file=file.path(path_analysis,'RealiabilityIntra.txt'), append=TRUE)
            capture.output(icc(data.frame(data.frame(x),data.frame(y)), model="twoway", type="agreement", unit="single"),
                           file=file.path(path_analysis,'RealiabilityIntra.txt'), append=TRUE)
            cat(paste0('------------ CCC'), file=file.path(path_analysis,'RealiabilityIntra.txt'), append=TRUE)            
            capture.output(CCC(x=x, y=y, ci="z-transform", conf.level=0.95, na.rm=FALSE),
                       file=file.path(path_analysis,'RealiabilityIntra.txt'), append=TRUE)
        }}}

Warning message in cor(dat$x, dat$y):
"l'écart type est nulle"
Warning message in cor(dat$x, dat$y):
"l'écart type est nulle"
Warning message in cor(dat$x, dat$y):
"l'écart type est nulle"


In [6]:
#categorical/binary data
for (person in unique(df[['Person']])){
    if (is.na(person)==FALSE){
        cat(paste0('--------------------------------------------------\n', person),
            file=file.path(path_analysis,'RealiabilityIntra.txt'), append=TRUE)
        for (var in c('Diarrhea', 'MissingToes')){
            cat(paste0('-----------------\n',var),file=file.path(path_analysis,'RealiabilityIntra.txt'), append=TRUE)
            df_ = data.frame(data.frame(subset(df, (df$PassageID =="1") & (df$Person==person))[[var]]),
                             data.frame(subset(df, (df$PassageID =="2") & (df$Person==person))[[var]]))
            capture.output(kappa2(df_[,c(1,2)], "unweighted"), 
                           file=file.path(path_analysis,'RealiabilityIntra.txt'), append=TRUE)
        }}}

# between Mark &cam - Inter observer reliability

In [7]:
#create a file to register data
sink(file.path(path_analysis,'RealiabilityInter.txt'))

In [8]:
#continuous data
for (passage in unique(df[['PassageID']])){
    if (is.na(person)==FALSE){
        cat(paste0('--------------------------------------------------\n', passage),
            file=file.path(path_analysis,'RealiabilityInter.txt'), append=TRUE)
        for (var in c_cont){
            cat(paste0('-----------------\n',var), file=file.path(path_analysis,'RealiabilityInter.txt'), append=TRUE)
            x = subset(df, (df$PassageID==passage) & (df$Person=="Mark"))[[var]]
            y = subset(df, (df$PassageID==passage) & (df$Person=="Camille"))[[var]]
            cat(paste0('------------ ICC'), file=file.path(path_analysis,'RealiabilityInter.txt'), append=TRUE)
            capture.output(icc(data.frame(data.frame(x),data.frame(y)), model = "twoway", type = "agreement", unit = "single"),
                           file=file.path(path_analysis,'RealiabilityInter.txt'), append=TRUE)
            cat(paste0('------------- CCC'), file=file.path(path_analysis,'RealiabilityInter.txt'), append=TRUE)
            capture.output(CCC(x=x, y=y, ci = "z-transform", conf.level = 0.95, na.rm = FALSE),
                           file=file.path(path_analysis,'RealiabilityInter.txt'), append=TRUE)
        }}}

Warning message in cor(dat$x, dat$y):
"l'écart type est nulle"
Warning message in cor(dat$x, dat$y):
"l'écart type est nulle"
Warning message in cor(dat$x, dat$y):
"l'écart type est nulle"
Warning message in cor(dat$x, dat$y):
"l'écart type est nulle"


In [9]:
#categorical/binary data
for (passage in unique(df[['PassageID']])){
    if (is.na(person)==FALSE){
        cat(paste0('--------------------------------------------------\n', passage),
            file=file.path(path_analysis,'RealiabilityInter.txt'), append=TRUE)
        for (var in c('Diarrhea', 'MissingToes')){
            cat(paste0('-----------------\n',var),file=file.path(path_analysis,'RealiabilityInter.txt'), append=TRUE)
            df_ = data.frame(data.frame(subset(df, (df$PassageID==passage) & (df$Person=="Mark"))[[var]]),
                             data.frame(subset(df, (df$PassageID==passage) & (df$Person=="Camille"))[[var]]))
            capture.output(kappa2(df_[,c(1,2)], "unweighted"), 
                           file=file.path(path_analysis,'RealiabilityInter.txt'), append=TRUE)
        }}}